In [1]:
import pandas as pd
import numpy as np

e7 = pd.read_excel('e7_filtered.xlsx')
g7 = pd.read_excel('g7_filtered.xlsx')

#head of the data

print(f"g7 data:")
g7.drop("Unnamed: 0", axis=1 , inplace=True)
e7.drop("Unnamed: 0" , axis=1 , inplace=True)
g7.head()

g7 data:


,year,country,inflation_rate,fdi_inflows,gdp,population_15_64,average_wages,robots,unemployment
0,2004,Canada,3.260874,1.452417e+09,1.026690e+12,69.121043,28326.60,302,7.185
1,2005,Canada,3.116639,2.555223e+10,1.173109e+12,69.316205,31624.08,305,6.758
2,2006,Canada,2.570173,6.430230e+10,1.319265e+12,69.461956,34606.56,311,6.484
3,2007,Canada,3.300546,1.204515e+11,1.468820e+12,69.523567,38130.00,323,6.156
4,2008,Canada,3.996316,7.011961e+10,1.552990e+12,69.514802,39463.44,317,6.284


In [2]:
e7.head()

,year,country,inflation_rate,fdi_inflows,gdp,population_65_and_above,average_wages,robots,unemployment
0,2004,China,6.951993,6.811727e+10,1.955347e+12,7.666347,1935.528,60,4.49
1,2005,China,3.903744,1.041087e+11,2.285961e+12,7.825417,2241.060,63,4.52
2,2006,China,3.926549,1.240820e+11,2.752119e+12,7.996802,2633.868,65,4.43
3,2007,China,7.749686,1.562493e+11,3.550328e+12,8.158964,3277.284,67,4.35
4,2008,China,7.795346,1.715347e+11,4.594337e+12,8.303532,4206.432,88,4.59


In [3]:
e7['id'] = np.repeat(np.arange(1, 8), 19)
g7['id'] = np.repeat(np.arange(1, 8), 19)

e7.head()

# print(e7.columns)
# print(g7.columns)

,year,country,inflation_rate,fdi_inflows,gdp,population_65_and_above,average_wages,robots,unemployment,id
0,2004,China,6.951993,6.811727e+10,1.955347e+12,7.666347,1935.528,60,4.49,1
1,2005,China,3.903744,1.041087e+11,2.285961e+12,7.825417,2241.060,63,4.52,1
2,2006,China,3.926549,1.240820e+11,2.752119e+12,7.996802,2633.868,65,4.43,1
3,2007,China,7.749686,1.562493e+11,3.550328e+12,8.158964,3277.284,67,4.35,1
4,2008,China,7.795346,1.715347e+11,4.594337e+12,8.303532,4206.432,88,4.59,1


In [4]:
e7.columns = ['year', 'country', 'inflation_rate', 'fdi_inflows', 'gdp',
       'population_65_and_above', 'average_wages', 'robots', 'unemployment',
       'id']
g7.columns = ['year', 'country', 'inflation_rate', 'fdi_inflows', 'gdp',
       'population_15_64', 'average_wages', 'robots', 'unemployment', 'id']


In [5]:
# eksik veri kontrolü
print(e7.isnull().sum())
print(g7.isnull().sum())

year                       0
country                    0
inflation_rate             0
fdi_inflows                0
gdp                        0
population_65_and_above    0
average_wages              0
robots                     0
unemployment               0
id                         0
dtype: int64
year                0
country             0
inflation_rate      0
fdi_inflows         0
gdp                 0
population_15_64    0
average_wages       0
robots              0
unemployment        0
id                  0
dtype: int64


In [6]:
# Sıfır veya negatif değerleri filtrelemek
e7 = e7[(e7['gdp'] > 0) & (e7['fdi_inflows'] > 0) & (e7['average_wages'] > 0)]
g7 = g7[(g7['gdp'] > 0) & (g7['fdi_inflows'] > 0) & (g7['average_wages'] > 0)]

# Logaritmik dönüşümler tekrar yapılabilir
e7['log_gdp'] = np.log(e7['gdp'])
e7['log_fdi_inflows'] = np.log(e7['fdi_inflows'])
e7['log_average_wages'] = np.log(e7['average_wages'])

g7['log_gdp'] = np.log(g7['gdp'])
g7['log_fdi_inflows'] = np.log(g7['fdi_inflows'])
g7['log_average_wages'] = np.log(g7['average_wages'])


In [7]:
from linearmodels import PanelOLS

Şimdi burada pesaran testi yapacağız

In [8]:
# Temel kütüphaneler
import pandas as pd
import numpy as np

# İstatistiksel analiz için
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Panel veri analizleri için
from linearmodels.panel import PooledOLS

# Pesaran'ın CD testi için
from linearmodels.panel import compare, PanelOLS
from scipy.stats import norm


In [9]:
def safe_log(series):
    return np.log(series.replace(0, np.nan)).dropna()

# E7 veri seti için
e7['log_average_wages'] = safe_log(e7['average_wages'])
e7['log_fdi_inflows'] = safe_log(e7['fdi_inflows'])
e7['log_gdp'] = safe_log(e7['gdp'])
e7['log_robots'] = safe_log(e7['robots'])
e7['log_unemployment'] = safe_log(e7['unemployment'])

# G7 veri seti için
g7['log_average_wages'] = safe_log(g7['average_wages'])
g7['log_fdi_inflows'] = safe_log(g7['fdi_inflows'])
g7['log_gdp'] = safe_log(g7['gdp'])
g7['log_robots'] = safe_log(g7['robots'])
g7['log_unemployment'] = safe_log(g7['unemployment'])


In [10]:
e7[['log_average_wages', 'log_fdi_inflows', 'log_gdp', 'log_robots', 'log_unemployment']].head()


,log_average_wages,log_fdi_inflows,log_gdp,log_robots,log_unemployment
0,7.568135,24.944497,28.301589,4.094345,1.501853
1,7.714704,25.368701,28.457808,4.143135,1.508512
2,7.876209,25.544209,28.643392,4.174387,1.488400
3,8.094770,25.774719,28.898061,4.204693,1.470176
4,8.344370,25.868051,29.155846,4.477337,1.523880


In [11]:
g7[['log_average_wages', 'log_fdi_inflows', 'log_gdp', 'log_robots', 'log_unemployment']].head()


,log_average_wages,log_fdi_inflows,log_gdp,log_robots,log_unemployment
0,10.251557,21.096495,27.657361,5.710427,1.971996
1,10.361674,23.963990,27.790678,5.720312,1.910727
2,10.451799,24.886861,27.908096,5.739793,1.869338
3,10.548757,25.514513,28.015481,5.777652,1.817427
4,10.583130,24.973468,28.071203,5.758902,1.838007


In [12]:
# Temel kütüphaneler
import pandas as pd
import numpy as np

# İstatistiksel analiz için
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Panel veri analizleri için
from linearmodels.panel import PooledOLS

# Pesaran'ın CD testi için
from linearmodels.panel import compare, PanelOLS
from scipy.stats import norm


In [13]:
# Eksik değerleri içeren satırları kaldırma
e7 = e7.dropna(subset=['log_average_wages', 'log_fdi_inflows', 'log_gdp', 'log_robots', 'log_unemployment'])
g7 = g7.dropna(subset=['log_average_wages', 'log_fdi_inflows', 'log_gdp', 'log_robots', 'log_unemployment'])


In [14]:
# uygun indeksleme

e7_panel = e7.set_index(['country', 'year'])

g7_panel = g7.set_index(['country', 'year'])


In [15]:
# E7 modeli
e7_formula = 'log_average_wages ~ log_fdi_inflows + log_gdp + log_robots + log_unemployment'
e7_model = PooledOLS.from_formula(e7_formula, data=e7_panel)
e7_results = e7_model.fit()
e7_results.summary

Dep. Variable:,log_average_wages,R-squared:,0.9959
Estimator:,PooledOLS,R-squared (Between):,0.9971
No. Observations:,132,R-squared (Within):,0.2108
Date:,"Wed, Aug 28 2024",R-squared (Overall):,0.9959
Time:,20:34:42,Log-likelihood,-104.64
Cov. Estimator:,Unadjusted,,
,,F-statistic:,7851.8
Entities:,7,P-value,0.0000
Avg Obs:,18.857,Distribution:,"F(4,128)"
Min Obs:,18.000,,
Max Obs:,19.000,F-statistic (robust):,7851.8


In [16]:


# G7 modeli
g7_formula = 'log_average_wages ~ log_fdi_inflows + log_gdp + log_robots + log_unemployment'
g7_model = PooledOLS.from_formula(g7_formula, data=g7_panel)
g7_results = g7_model.fit()
g7_results.summary


Dep. Variable:,log_average_wages,R-squared:,0.9994
Estimator:,PooledOLS,R-squared (Between):,0.9996
No. Observations:,127,R-squared (Within):,-0.0439
Date:,"Wed, Aug 28 2024",R-squared (Overall):,0.9994
Time:,20:34:42,Log-likelihood,-7.8335
Cov. Estimator:,Unadjusted,,
,,F-statistic:,5.218e+04
Entities:,7,P-value,0.0000
Avg Obs:,18.143,Distribution:,"F(4,123)"
Min Obs:,17.000,,
Max Obs:,19.000,F-statistic (robust):,5.218e+04


Pasaran cross depandence testi için hazır fonksiyon setup ı

In [17]:
def pesaran_cd(residuals):
    """
    Pesaran's Cross-sectional Dependence (CD) Test
    """
    residuals = residuals.unstack(level=0)  # Her birimi ayrı bir sütun olarak düzenle
    T, N = residuals.shape
    sum_correlations = 0
    for i in range(N):
        for j in range(i+1, N):
            corr = np.corrcoef(residuals.iloc[:, i], residuals.iloc[:, j])[0, 1]
            sum_correlations += corr
    avg_correlation = (2 / (N * (N - 1))) * sum_correlations
    test_stat = np.sqrt(N * (N - 1) / (2 * T)) * avg_correlation
    p_value = 2 * (1 - norm.cdf(abs(test_stat)))
    return {'Test Statistic': test_stat, 'p-value': p_value, 'Average Correlation': avg_correlation}


In [18]:
# E7 modeli için
e7_cd_test = pesaran_cd(e7_results.resids)
print("Pesaran's CD Test Results for E7 Model:")
print(e7_cd_test)

# G7 modeli için
g7_cd_test = pesaran_cd(g7_results.resids)
print("Pesaran's CD Test Results for G7 Model:")
print(g7_cd_test)


Pesaran's CD Test Results for E7 Model:
{'Test Statistic': np.float64(nan), 'p-value': np.float64(nan), 'Average Correlation': np.float64(nan)}
Pesaran's CD Test Results for G7 Model:
{'Test Statistic': np.float64(nan), 'p-value': np.float64(nan), 'Average Correlation': np.float64(nan)}
